In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

import setting

path_dict = path(setting)

In [ ]:
w = pd.read_table(
    path_dict['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    path_dict['h_file_path'],
    index_col=0,
)

In [ ]:
nmf_map = ccal.NMFMap(
    wt=w.T,
    h=h,
    mds_random_seed=setting.NMF_MAP_MDS_RANDOM_SEED,
    wt_pull_power=setting.NMF_MAP_WT_PULL_POWER,
    h_pull_power=setting.NMF_MAP_H_PULL_POWER,
    plot=False,
)

In [ ]:
for wt_or_h, df, bandwidth_factor, element_marker_size in (
    (
        'wt',
        pd.DataFrame(
            nmf_map.wt,
            index=nmf_map.nodes,
            columns=nmf_map.wt_elements,
        ),
        setting.NMF_MAP_WT_BANDWIDTH_FACTOR,
        setting.NMF_MAP_WT_ELEMENT_MARKER_SIZE,
    ),
    (
        'h',
        pd.DataFrame(
            nmf_map.h,
            index=nmf_map.nodes,
            columns=nmf_map.h_elements,
        ),
        setting.NMF_MAP_H_BANDWIDTH_FACTOR,
        setting.NMF_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):

    if df.shape[1] < 1e3:

        distance_function = ccal.compute_information_distance

    else:

        distance_function = compute_correlation_distance

    hcc__k_x_column = ccal.hierarchical_consensus_cluster_with_multiple_k(
        df,
        setting.HCC_KS,
        n_job=setting.MAX_N_JOB,
        distance_function=distance_function,
        plot_df=False,
        directory_path=path_dict['{}_hcc_directory_path'.format(wt_or_h)],
    )[3]

    hcc__k_x_column = pd.read_table(
        path_dict['{}_hcc__k_x_column_file_path'.format(wt_or_h)],
        index_col=0,
    )

    for hcc_k in setting.HCC_KS:

        element_state = hcc__k_x_column.loc['K{}'.format(hcc_k)]

        if element_state.value_counts().min() < 3:

            print('Skipping {} because it has at least 1 state with less than 3 elements ...'.format(hcc_k))

        else:

            nmf_map.set_element_states(
                wt_or_h,
                element_state,
                bandwidth_factor=bandwidth_factor,
            )

            nmf_map.plot_nmf_map(
                wt_or_h,
                title=setting.NAME,
                element_marker_size=element_marker_size,
                html_file_path='{}/map_{}.html'.format(
                    path_dict['{}_map_directory_path'.format(wt_or_h)],
                    hcc_k,
                ),
            )